<a target="_blank" href="https://colab.research.google.com/github/SydanJainen/AlgorithmsForMassiveDataset/blob/main/Lesson2/lesson2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

Impossibile trovare il percorso specificato.
"wget" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.
tar: Error opening archive: Failed to open 'spark-2.4.5-bin-hadoop2.7.tgz'


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [3]:
import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Exception: Unable to find py4j in spark-2.4.5-bin-hadoop2.7\python, your SPARK_HOME may not be configured correctly

In [ ]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [ ]:
sc = spark.sparkContext

In [ ]:
s = '''Every person had a star, every star had a friend, and for every person
carrying a star there was someone else who reflected it, and everyone
carried this reflection like a secret confidante in the heart'''
simple_rdd = sc.parallelize(s.split('\n'))

In [ ]:
simple_rdd.collect()

['Every person had a star, every star had a friend, and for every person',
 'carrying a star there was someone else who reflected it, and everyone',
 'carried this reflection like a secret confidante in the heart']

In [ ]:
(simple_rdd.map(lambda line: line.split(' '))
           .collect())

[['Every',
  'person',
  'had',
  'a',
  'star,',
  'every',
  'star',
  'had',
  'a',
  'friend,',
  'and',
  'for',
  'every',
  'person'],
 ['carrying',
  'a',
  'star',
  'there',
  'was',
  'someone',
  'else',
  'who',
  'reflected',
  'it,',
  'and',
  'everyone'],
 ['carried',
  'this',
  'reflection',
  'like',
  'a',
  'secret',
  'confidante',
  'in',
  'the',
  'heart']]

In [ ]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .take(10))

['Every',
 'person',
 'had',
 'a',
 'star,',
 'every',
 'star',
 'had',
 'a',
 'friend,']

In [ ]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .map(lambda word: word.replace(',', '').lower())
           .take(5))

['every', 'person', 'had', 'a', 'star']

In [ ]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .map(lambda word: word.replace(',', '').lower())
           .map(lambda word: (word, 1))
           .take(5))

[('every', 1), ('person', 1), ('had', 1), ('a', 1), ('star', 1)]

In [ ]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .map(lambda word: word.replace(',', '').lower())
           .map(lambda word: (word, 1))
           .reduceByKey(lambda a,b: a+b)
           .collect())

[('person', 2),
 ('there', 1),
 ('was', 1),
 ('carried', 1),
 ('this', 1),
 ('like', 1),
 ('secret', 1),
 ('confidante', 1),
 ('in', 1),
 ('heart', 1),
 ('every', 3),
 ('had', 2),
 ('a', 4),
 ('star', 3),
 ('friend', 1),
 ('and', 2),
 ('for', 1),
 ('carrying', 1),
 ('someone', 1),
 ('else', 1),
 ('who', 1),
 ('reflected', 1),
 ('it', 1),
 ('everyone', 1),
 ('reflection', 1),
 ('the', 1)]

In [ ]:
def count_freq(rdd):
  return (rdd.flatMap(lambda line: line.split(' '))
            .map(lambda word: word.replace(',', '').lower())
            .map(lambda word: (word, 1))
            .reduceByKey(lambda a,b: a+b)
            .collect())

In [ ]:
!wget http://www.scifiscripts.com/scripts/swd1_5-74.txt

--2020-03-11 11:24:00--  http://www.scifiscripts.com/scripts/swd1_5-74.txt
Resolving www.scifiscripts.com (www.scifiscripts.com)... 207.32.177.145
Connecting to www.scifiscripts.com (www.scifiscripts.com)|207.32.177.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203125 (198K) [text/plain]
Saving to: â€˜swd1_5-74.txt.1â€™

swd1_5-74.txt.1     100%[===================>] 198.36K   294KB/s    in 0.7s    

2020-03-11 11:24:01 (294 KB/s) - â€˜swd1_5-74.txt.1â€™ saved [203125/203125]



In [ ]:
sw = sc.textFile('swd1_5-74.txt')

In [ ]:
sw.take(10)

['The Star Wars',
 'by',
 'George Lucas',
 '',
 '',
 '',
 'Rough Draft [First of four major screenplay drafts]',
 'Lucasfilm Ltd.',
 '5/74',
 '']

In [ ]:
count_freq(sw)[: 10]

[('george', 1),
 ('', 3543),
 ('draft', 1),
 ('of', 744),
 ('four', 25),
 ('major', 2),
 ('drafts]', 1),
 ('ltd.', 1),
 ('5/74', 1),
 ('1.', 1)]

In [ ]:
import random

NUM_SAMPLES = 10**7

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = (sc.parallelize(range(0, NUM_SAMPLES))
           .filter(inside).count())

print('Pi is roughly {}'.format(4.0 * count / NUM_SAMPLES))

Pi is roughly 3.1404776
